In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,42786
2,Huelva,Confirmados PDIA 14 días,2457
3,Huelva,Tasa PDIA 14 días,"478,78870549720364"
4,Huelva,Confirmados PDIA 7 días,848
5,Huelva,Tasa PDIA 7 dias,"165,24738390786678"
6,Huelva,Total Confirmados,42985
7,Huelva,Curados,36082
8,Huelva,Fallecidos,397


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  42786.0


/tmp/ipykernel_3309/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  11944.0


/tmp/ipykernel_3309/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_3309/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_3309/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_3309/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 620 personas en los últimos 7 días 

Un positivo PCR cada 202 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,42786.0,848.0,2457.0,513170.0,165.247384,478.788705,230.0
Huelva-Costa,25186.0,512.0,1490.0,289548.0,176.827331,514.595162,147.0
Huelva (capital),11944.0,232.0,711.0,143837.0,161.293687,494.309531,66.0
Condado-Campiña,13418.0,260.0,773.0,156231.0,166.420237,494.780165,62.0
Ayamonte,1886.0,73.0,169.0,21104.0,345.905989,800.796058,34.0
Sierra de Huelva-Andévalo Central,3788.0,68.0,181.0,67391.0,100.903682,268.581858,18.0
Cartaya,1728.0,53.0,112.0,20083.0,263.904795,557.685605,13.0
Bollullos Par del Condado,1238.0,30.0,95.0,14387.0,208.521582,660.318343,13.0
Isla Cristina,2894.0,55.0,156.0,21393.0,257.093442,729.210489,11.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Sanlúcar de Guadiana,31.0,1.0,12.0,403.0,248.138958,2977.667494,0.0
Cañaveral de León,21.0,6.0,8.0,396.0,1515.151515,2020.202020,3.0
Alájar,27.0,7.0,9.0,759.0,922.266140,1185.770751,2.0
Corteconcepción,44.0,1.0,6.0,536.0,186.567164,1119.402985,NaN
Escacena del Campo,207.0,3.0,22.0,2287.0,131.176213,961.958898,0.0
Villarrasa,215.0,2.0,21.0,2211.0,90.456807,949.796472,1.0
Campofrío,20.0,3.0,6.0,713.0,420.757363,841.514727,1.0
Bonares,438.0,14.0,50.0,6060.0,231.023102,825.082508,2.0
Paterna del Campo,190.0,11.0,28.0,3457.0,318.194967,809.950824,2.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Sanlúcar de Guadiana,31.0,1.0,12.0,403.0,248.138958,2977.667494,0.0,0.083333
Villarrasa,215.0,2.0,21.0,2211.0,90.456807,949.796472,1.0,0.095238
Escacena del Campo,207.0,3.0,22.0,2287.0,131.176213,961.958898,0.0,0.136364
Campillo (El),98.0,2.0,13.0,2024.0,98.814229,642.292490,0.0,0.153846
Corteconcepción,44.0,1.0,6.0,536.0,186.567164,1119.402985,0.0,0.166667
Villalba del Alcor,492.0,1.0,6.0,3366.0,29.708853,178.253119,0.0,0.166667
Lucena del Puerto,343.0,4.0,18.0,3261.0,122.661760,551.977921,0.0,0.222222
Valverde del Camino,803.0,6.0,26.0,12750.0,47.058824,203.921569,1.0,0.230769
Lepe,2819.0,42.0,173.0,27880.0,150.645624,620.516499,9.0,0.242775
